In [1]:
pip install -q folium

In [ ]:
!pip install -q geopandas

In [ ]:
!pip install -q pandas-profiling==2.7.1

In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon
import geopandas as gpd
from pandas_profiling import ProfileReport

In [5]:
jeff_vlc = pd.read_csv('jeff_vlc_poi.csv', sep=',', decimal='.')
jeff_vlc.drop(['Unnamed: 0','Unnamed: 0.1','partner_id','prtn_address_latitude','prtn_address_longitude','order_id', 'Dist_Cliente_Tienda'],inplace=True,axis=1)

In [6]:
dataset_ine = pd.read_csv('dataset_ine_valencia.csv', sep=',', decimal='.')
dataset_ine.drop(['coddistrit','codsec','Unnamed: 0'],inplace=True,axis=1)

In [7]:
with open('seccensales.JSON') as f:
    seccensales_geojson = json.load(f)

In [8]:
def DrawLaundry_Points(latitude, longitude, map,string):
    folium.Marker(location=[latitude,longitude ],popup=string).add_to( map )

In [9]:
def get_seccensal(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures["features"]:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return int(seccensal)

In [10]:
def draw_map_cluster(dataset):
  map = folium.Map(location=[39.46994829189022, -0.37787440832473984], tiles='CartoDB positron', zoom_start=14)
  marker_cluster = MarkerCluster().add_to(map)
  dataset_visual = dataset
  dataset_visual = dataset_visual.drop_duplicates('customer_id')
  dataset_visual.apply(lambda row: DrawLaundry_Points(row['ord_billing_address_lat'],row['ord_billing_address_long'],marker_cluster,row['prtn_code']), axis=1)
  return map

In [11]:
jeff_vlc["Code"] = jeff_vlc.apply(lambda row: get_seccensal(row["ord_billing_address_lat"], row["ord_billing_address_long"], seccensales_geojson), axis=1) 
jeff_vlc = jeff_vlc[jeff_vlc["Code"]!= 0]

In [12]:
jeff_vlc = pd.merge(jeff_vlc, dataset_ine, on="Code", how="inner")

Pedidos de las dos modalidades

In [ ]:
map_total = draw_map_cluster(jeff_vlc)
map_total

Solo pedidos offline

In [ ]:
jeff_vlc_offline=jeff_vlc[jeff_vlc['ord_channel_type_code']=='OFFLINE']
map_total_offline = draw_map_cluster(jeff_vlc_offline)
map_total_offline

In [ ]:
jeff_vlc_online=jeff_vlc[jeff_vlc['ord_channel_type_code']=='ONLINE']
map_total_online = draw_map_cluster(jeff_vlc_online)
map_total_online

**Tienda petxina**

Canal Offline Petxina

In [ ]:
jeff_vlc_offline_petxina=jeff_vlc_offline[jeff_vlc_offline['prtn_code']=='PETXINA']
map_total_offline_petxina = draw_map_cluster(jeff_vlc_offline_petxina)
map_total_offline_petxina

Canal Online Petxina

In [ ]:
jeff_vlc_online_petxina=jeff_vlc_online[jeff_vlc_online['prtn_code']=='PETXINA']
map_total_online_petxina = draw_map_cluster(jeff_vlc_online_petxina)
map_total_online_petxina

**Tienda Reino**

Canal Offline Reino

In [ ]:
jeff_vlc_offline_reino=jeff_vlc_offline[jeff_vlc_offline['prtn_code']=='REINO']
map_total_offline_reino = draw_map_cluster(jeff_vlc_offline_reino)
map_total_offline_reino

Canal Online Reino

In [ ]:
jeff_vlc_online_reino=jeff_vlc_online[jeff_vlc_online['prtn_code']=='REINO']
map_total_online_reino = draw_map_cluster(jeff_vlc_online_reino)
map_total_online_reino

**Correlaciones valencia**

Pandas Profile

In [ ]:
report_vlc_online = ProfileReport(jeff_vlc_online)
report_vlc_online.to_file("jeff_vlc_online_report.html")

In [ ]:
report_vlc_offline = ProfileReport(jeff_vlc_offline)
report_vlc_offline.to_file("jeff_vlc_offline_report.html")

In [ ]:
report_vlc_online_petxina = ProfileReport(jeff_vlc_online_petxina)
report_vlc_online_petxina.to_file("jeff_vlc_online_petxina_report.html")

In [ ]:
report_vlc_offline_petxina = ProfileReport(jeff_vlc_offline_petxina)
report_vlc_offline_petxina.to_file("jeff_vlc_offline_petxina_report.html")

In [ ]:
report_vlc_online_reino = ProfileReport(jeff_vlc_online_reino)
report_vlc_online_reino.to_file("jeff_vlc_online_reino_report.html")

In [ ]:
report_vlc_offline_reino = ProfileReport(jeff_vlc_offline_reino)
report_vlc_offline_reino.to_file("jeff_vlc_offline_reino_report.html")

Correlacion

In [22]:
def get_correlation_shop(in_vlc):
  customer_id_count = in_vlc[['customer_id','total_poi']]
  customer_id_count = customer_id_count.groupby(by=['customer_id']).count()
  customer_id_count=customer_id_count.reset_index(level='customer_id')
  customer_id_count = customer_id_count.rename(columns={'total_poi': 'num_operaciones'})
  result = pd.merge(customer_id_count, in_vlc, on="customer_id")
  result_final = result.drop_duplicates('customer_id')
  result_final.reset_index(inplace=True)
  result_final.drop(['customer_id','index','prtn_code','prtn_city_code','ord_channel_code','ord_channel_type_code','ord_logistics_code','ord_creation_date_time','Code', 'ord_billing_address_lat', 'ord_billing_address_long'],inplace=True,axis=1)
  resultados_correlacion = result_final.corr()
  result_corr = resultados_correlacion['num_operaciones']
  return result_corr

Correlacion Petxina offline y online

In [ ]:
result_petxina_online = get_correlation_shop(jeff_vlc_online_petxina)
result_petxina_offline = get_correlation_shop(jeff_vlc_offline_petxina)

In [ ]:
result_petxina_online

In [ ]:
result_petxina_offline

Correlacion Reino offline y online

In [ ]:
result_reino_online = get_correlation_shop(jeff_vlc_online_reino)
result_reino_offline = get_correlation_shop(jeff_vlc_offline_reino)

In [ ]:
result_reino_online

In [ ]:
result_reino_offline

In [ ]:
jeff_vlc_online_petxina

Correlacion Datos INE

In [31]:
jeff_online_petxina_ine = jeff_vlc_online_petxina.drop(['education','lavanderias','tiendas','transporte','gimnasios','parques','restaurante','bancos','estadios', 'hospitales', 'total_poi','Code'],axis=1)
jeff_offline_petxina_ine = jeff_vlc_offline_petxina.drop(['education','lavanderias','tiendas','transporte','gimnasios','parques','restaurante','bancos','estadios', 'hospitales', 'total_poi','Code'],axis=1)
jeff_online_reino_ine = jeff_vlc_online_reino.drop(['education','lavanderias','tiendas','transporte','gimnasios','parques','restaurante','bancos','estadios', 'hospitales', 'total_poi','Code'],axis=1)
jeff_offline_reino_ine = jeff_vlc_offline_reino.drop(['education','lavanderias','tiendas','transporte','gimnasios','parques','restaurante','bancos','estadios', 'hospitales', 'total_poi','Code'],axis=1)
result_jeff_online_petxina_ine =jeff_online_petxina_ine.corr()
result_jeff_offline_petxina_ine =jeff_offline_petxina_ine.corr()
result_jeff_online_reino_ine =jeff_online_reino_ine.corr()
result_jeff_offline_reino_ine =jeff_offline_reino_ine.corr()   

In [ ]:
result_jeff_online_petxina_ine

In [ ]:
result_jeff_offline_petxina_ine

In [ ]:
result_jeff_online_reino_ine

In [ ]:
result_jeff_offline_reino_ine